In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -r /content/drive/MyDrive/2021_s1/Jianeng_test/requirements.txt

  Created wheel for Node2Vec: filename=node2vec-0.4.2-cp37-none-any.whl size=5859 sha256=638d935fa49c7f31745b531271a3633b7020b02d031c16d3cac49f254f7e8c93
  Stored in directory: /root/.cache/pip/wheels/2a/d6/34/7f62d314dfad813eea2a41c5e4a9dedec1f52e335688c8f7ad
Successfully built Node2Vec


In [3]:
import pandas as pd
import numpy 
import networkx as net
import pandas as pd
import numpy as np
import random 
from node2vec import Node2Vec
import tqdm

In [4]:
# FielName="preprocessed/graphData.txt"
FielName="/content/drive/MyDrive/2021_s1/dataset/edges_with_bade_cases.txt" # 包括了test public内traintxt没有的边（自己与自己链接
Graphtype=net.Graph()   # use net.Graph() for undirected graph

# How to read from a file. Note: if your egde weights are int, 
# change float to int.
G = net.read_edgelist(
    FielName, 
    create_using=Graphtype,
    nodetype=int,
    data=(('weight',float),)
)

# # Find the total number of degree, in_degree and out_degree for each node
# for x in G.nodes():
#     print(
#         "Node: ", x, " has total #degree: ",G.degree(x) 
#     )

# Find the weight for each node
# for u,v in G.edges():
#       print ("Weight of Edge ("+str(u)+","+str(v)+")", G.get_edge_data(u,v))

In [ ]:
# statistic feature test

In [ ]:
# # 保存模型
# from gensim.models import Word2Vec
# model.wv.save("/content/drive/MyDrive/2021_s1_test_sml/model/model.wv") 

In [ ]:
# model.wv.save_word2vec_format("/content/drive/MyDrive/2021_s1_test_sml/model/model.bin", binary=True)

In [ ]:
# 加载模型
# from gensim.models import KeyedVectors
# wv = KeyedVectors.load("/content/drive/MyDrive/2021_s1_test_sml/model/model.wv", mmap='r')
# # vector = wv['computer'] # numpy vector of a word
# #  https://juejin.cn/post/6844903912256831502

In [ ]:
# model = wv

In [ ]:
#  model['1864']

In [ ]:
# # Generating input vectors
# def generate_final_dataset(model,file_name):
      
#     X, Y = [], []
#     with open(file_name, 'r') as f: 
#         for line in f: 
#             pre_node, next_node, y = line.split()
#             print(pre_node, next_node, y)
#             X_vec = list(model[str(pre_node)])
#             X_vec.extend(model[str(next_node)])
#             X.append(X_vec)
#             Y.append(int(y))
#     return X, Y
 

In [ ]:
#  len(sorted(net.common_neighbors(G, 356, 0)))

7

In [5]:
 # common_ne, n_shortest_path, jaccard_coefficient_p, ra and aa
# Generating input vectors using features
def generate_final_dataset_using_statistic_feature( file_name):
      
    X, Y = [], []
    with open(file_name, 'r') as f: 
        for line in f: 
            pre_node, next_node, y = line.split() 
            try:
              common_ne = len(sorted(net.common_neighbors(G, int(pre_node), int(next_node))))
            except:
              common_ne = len(sorted(net.common_neighbors(G, int(next_node), int(pre_node))))

            try:
              try:
                n_shortest_path = len( (net.shortest_path(G, int(pre_node), int(next_node))))
              except:
                n_shortest_path = len( (net.shortest_path(G, int(next_node), int(pre_node))))     
            except:
              n_shortest_path = 0

            jaccard_coefficient_p = None
            try:
              jaccard_coefficient_ps = net.jaccard_coefficient(G,[(int(pre_node), int(next_node))])  

              for u, v, p in jaccard_coefficient_ps:
                jaccard_coefficient_p = p
            except:
              jaccard_coefficient_ps = net.jaccard_coefficient(G,[(int(next_node), int(pre_node))])  
              for u, v, p in jaccard_coefficient_ps:
                jaccard_coefficient_p = p
 
            ra = None
            try:
              resource_allocations = net.resource_allocation_index(G,[(int(pre_node), int(next_node))])  

              for u, v, p in resource_allocations:
                ra = p
            except:
              resource_allocations = net.resource_allocation_index(G,[(int(next_node), int(pre_node))])  
              for u, v, p in resource_allocations:
                ra = p 

            # pa = None
            # try:
            #   preferential_attachments = net.preferential_attachment(G,[(int(pre_node), int(next_node))])  

            #   for u, v, p in preferential_attachments:
            #     pa = p
            # except:
            #   preferential_attachments = net.preferential_attachment(G,[(int(next_node), int(pre_node))])  
            #   for u, v, p in preferential_attachments:
            #     pa = p


            aa = None
            try:
              adamic_adar_indexs = net.adamic_adar_index(G,[(int(pre_node), int(next_node))])  

              for u, v, p in adamic_adar_indexs:
                aa = p
            except:
              adamic_adar_indexs = net.adamic_adar_index(G,[(int(next_node), int(pre_node))])  
              for u, v, p in adamic_adar_indexs:
                aa = p

            X_vec = []
            X_vec.append(common_ne)
            X_vec.append(n_shortest_path)
            X_vec.append(jaccard_coefficient_p)
            X_vec.append(ra)
            # X_vec.append(pa)
            X_vec.append(aa)
            # X_emb = (list(model[str(pre_node)]))
            # X_emb.extend(model[str(next_node)]) 
            # X_vec = X_vec + X_emb
            X.append(X_vec)
             
            Y.append(int(y))
    return X, Y

In [ ]:
# from sklearn.model_selection import train_test_split
# import pickle
# import pandas as pd
# from sklearn.metrics import classification_report

# # Xtrain, Ytrain =  generate_final_dataset(model,'/content/drive/MyDrive/2021_s1/preprocessed/train_data.txt')
# # Xtest, Ytest =  generate_final_dataset(model,'/content/drive/MyDrive/2021_s1/preprocessed/test2.txt')
# X, Y =  generate_final_dataset(model,'/content/drive/MyDrive/2021_s1_test_sml/preprocessed/trainingset10.txt') # 0.8的全边数据，随机提取了0.2数据的点组成负样本，而正样本在0.2内不存在
# # Spliting data into training set and testing set
# Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, stratify=Y, random_state=1,train_size=0.8, test_size = 0.2)

In [6]:
## new
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from sklearn.metrics import classification_report

# Xtrain, Ytrain =  generate_final_dataset(model,'/content/drive/MyDrive/2021_s1/preprocessed/train_data.txt')
# Xtest, Ytest =  generate_final_dataset(model,'/content/drive/MyDrive/2021_s1/preprocessed/test2.txt')
X, Y =  generate_final_dataset_using_statistic_feature( '/content/drive/MyDrive/2021_s1/Jianeng_test/preprocessed/trainingset10.txt') # 0.8的全边数据，随机提取了0.2数据的点组成负样本，而正样本在0.2内不存在
# Spliting data into training set and testing set
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, stratify=Y, random_state=1,train_size=0.8, test_size = 0.2)
 

In [ ]:
count = 0
for ele in Ytest:
  if ele is 1:
    count +=1
count

3207

In [ ]:
count = 0
for ele in Ytest:
  if ele == 0:
    count +=1
count

3207

In [ ]:
Xtrain

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
Xtrain = sc.fit_transform(Xtrain)
Xtest = sc.transform(Xtest)

In [8]:
Xtrain

array([[-0.62745802, -0.74850354, -0.41745697, -0.44573017, -0.55088276],
       [ 0.40937342, -0.31940611,  0.09428845, -0.06462688,  0.16412411],
       [ 2.4830363 , -0.74850354,  0.93716328,  4.31228663,  3.62297237],
       ...,
       [-0.62745802,  1.39698359, -0.41745697, -0.44573017, -0.55088276],
       [ 0.40937342, -0.31940611,  2.14127017, -0.15257379,  0.10654686],
       [-0.62745802,  1.39698359, -0.41745697, -0.44573017, -0.55088276]])

In [9]:
# Changing data to ndarray format
import numpy as np
Xtrain = np.array(Xtrain)
Xtest = np.array(Xtest)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X = sc.fit_transform(X) 
# # Changing data to ndarray format
# import numpy as np
# X = np.array(X) 

In [ ]:
len(Xtrain)

25654

In [ ]:
Xtest.shape

(6414, 5)

In [10]:
# Setting LR model and training the model
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver = 'sag')
log_reg.fit(Xtrain, Ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
# Making prediction on the testing set
prediction = log_reg.predict(Xtest)

In [12]:
# Evaluating the model
report = classification_report(Ytest, prediction, zero_division = 0, digits = 4)

from sklearn.metrics import roc_curve, auc, roc_auc_score
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytest, prediction)
auc = auc(false_positive_rate, true_positive_rate)

report += ('auc = ' + str(auc))
print(report)

              precision    recall  f1-score   support

           0     0.8282    0.9953    0.9041      3207
           1     0.9941    0.7936    0.8826      3207

    accuracy                         0.8944      6414
   macro avg     0.9112    0.8944    0.8934      6414
weighted avg     0.9112    0.8944    0.8934      6414
auc = 0.894449641409417


In [10]:
# Thanks to: https://www.kaggle.com/hhllcks/neural-net-with-gridsearch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import neural_network
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score,precision_score,recall_score,roc_auc_score,accuracy_score,roc_curve
%matplotlib inline

from sklearn import neural_network
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 

In [ ]:
np.arange(0,1)

array([0])

In [ ]:
from sklearn import neural_network
# 10.23 not original experiment
parameters = {'solver': ['adam'], 'max_iter': [500,1000,  1500,   2000 ], 'alpha': 10.0 ** -np.arange(-1, 4), 'hidden_layer_sizes':np.arange(10,20,2), 'random_state':[1]}
# parameters = {'solver': ['adam'], 'max_iter': [500,1000,  1600, 2000 ], 'alpha': 10.0 ** -np.arange(0, 2), 'hidden_layer_sizes':np.arange(15,21,2), 'random_state':[1]}

nn_grid = GridSearchCV(neural_network.MLPClassifier(), parameters,n_jobs=-1,cv=5,scoring='roc_auc')



grid_result=nn_grid.fit(Xtrain, Ytrain)
print("Best: %f using %s" % (grid_result.best_score_,nn_grid.best_params_))
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

Best: 0.998369 using {'alpha': 0.001, 'hidden_layer_sizes': 18, 'max_iter': 500, 'random_state': 1, 'solver': 'adam'}
0.955820  with:   {'alpha': 10.0, 'hidden_layer_sizes': 10, 'max_iter': 500, 'random_state': 1, 'solver': 'adam'}
0.955820  with:   {'alpha': 10.0, 'hidden_layer_sizes': 10, 'max_iter': 1000, 'random_state': 1, 'solver': 'adam'}
0.955820  with:   {'alpha': 10.0, 'hidden_layer_sizes': 10, 'max_iter': 1500, 'random_state': 1, 'solver': 'adam'}
0.955820  with:   {'alpha': 10.0, 'hidden_layer_sizes': 10, 'max_iter': 2000, 'random_state': 1, 'solver': 'adam'}
0.955822  with:   {'alpha': 10.0, 'hidden_layer_sizes': 12, 'max_iter': 500, 'random_state': 1, 'solver': 'adam'}
0.955822  with:   {'alpha': 10.0, 'hidden_layer_sizes': 12, 'max_iter': 1000, 'random_state': 1, 'solver': 'adam'}
0.955822  with:   {'alpha': 10.0, 'hidden_layer_sizes': 12, 'max_iter': 1500, 'random_state': 1, 'solver': 'adam'}
0.955822  with:   {'alpha': 10.0, 'hidden_layer_sizes': 12, 'max_iter': 2000, '

In [ ]:
# validation set - get optimal parameters
# cv- random forest
from sklearn.ensemble import RandomForestClassifier 

n_estimators = range(10,30,20)
random_state=[1]
max_depth=[3]
criterion=["entropy"]
min_samples_split=range(10,30,20)
param_grid = dict(n_estimators=n_estimators , max_depth=max_depth,criterion=criterion,min_samples_split=min_samples_split,random_state=random_state)
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid,  scoring='roc_auc', cv=10)
grid_result_rf = grid_search_rf.fit(Xtrain, Ytrain)
print("Best: %f using %s" % (grid_result_rf.best_score_,grid_search_rf.best_params_))
means = grid_result_rf.cv_results_['mean_test_score']
params = grid_result_rf.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

Best: 0.998364 using {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 10, 'random_state': 1}
0.998364  with:   {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 10, 'n_estimators': 10, 'random_state': 1}


In [11]:
from xgboost import XGBClassifier

parameters = {'n_estimators': range(30,120,40),'learning_rate':[0.05, 0.1, 0.15],
             'max_depth':range(1,4,2),'random_state':[1] ,'min_child_weight':range(1,4,2), 
#               'reg_lambda':[0.6,0.7],'reg_alpha':[0]
             }
grid_search_xgb = GridSearchCV(XGBClassifier(), parameters, n_jobs=-1,cv=5,
                   scoring='roc_auc' )
grid_result_xgb = grid_search_xgb.fit(Xtrain, Ytrain)
print("Best: %f using %s" % (grid_result_xgb.best_score_,grid_search_xgb.best_params_))
means = grid_result_xgb.cv_results_['mean_test_score']
params = grid_result_xgb.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

Best: 0.998483 using {'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 110, 'random_state': 1}
0.995071  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 30, 'random_state': 1}
0.995071  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 70, 'random_state': 1}
0.995071  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 110, 'random_state': 1}
0.995071  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 30, 'random_state': 1}
0.995071  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 70, 'random_state': 1}
0.995071  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 110, 'random_state': 1}
0.997953  with:   {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 30, 'random_state': 1}
0.997953  with:   {'learning_rate':

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
# GBDT
parameters = {
    "learning_rate": [0.02,0.03,0.05,0.07,0.1],
    "subsample":[0.1,0.2,0.3,0.6],
    "max_depth":range(1,6,1),
    'min_samples_split':range(2,5,1),
    'random_state':[1]}
grid_search_GBDT = GridSearchCV(GradientBoostingClassifier(), parameters, n_jobs=-1,cv=2, scoring='roc_auc' )
grid_result_GBDT = grid_search_GBDT.fit(Xtrain, Ytrain)
print("Best: %f using %s" % (grid_result_GBDT.best_score_,grid_search_GBDT.best_params_))
means = grid_result_GBDT.cv_results_['mean_test_score']
params = grid_result_GBDT.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

Best: 0.998548 using {'learning_rate': 0.1, 'max_depth': 2, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.6}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.1}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.2}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.3}
0.995073  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.6}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 3, 'random_state': 1, 'subsample': 0.1}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 3, 'random_state': 1, 'subsample': 0.2}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 3, 'random_state': 1, 'subsample': 0.3}
0.995073  with:   {'learning_rate': 0.02, 'max

In [ ]:
clf_1=RandomForestClassifier(criterion='entropy',max_depth= (3), n_estimators=(10),min_samples_split=10 ) #original


In [ ]:
# baseline
clf_1=neural_network.MLPClassifier( alpha=0.1,   hidden_layer_sizes=10, max_iter=500, solver='adam', random_state=1) # lb 82.24       


In [13]:
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
# clf_1=DecisionTreeClassifier(max_depth=(8), min_samples_split=33,random_state=1)
# clf_1=GaussianNB()

# dtc = DecisionTreeClassifier(criterion='gini',splitter='random',max_depth=2,min_samples_split=2,random_state=1)
# clf_1=AdaBoostClassifier(base_estimator=dtc,n_estimators=(4))
# clf_2=GradientBoostingClassifier(learning_rate=(0.1),max_depth=(4), subsample=0.3,random_state=1) 
clf_2=XGBClassifier( n_estimators= (110),learning_rate=(0.05), max_depth=(1), random_state=1, min_child_weight=1)
# clf_1=XGBClassifier(colsample_bytree=0.2,gamma=0.2,n_estimators= (80),learning_rate=(0.04), max_depth=(3),min_child_weight=2,num_parallel_tree=3,random_state=1,subsample=0.3)
clf_3=GradientBoostingClassifier(learning_rate=(0.1),max_depth=(2), min_samples_split=2,subsample=0.6,random_state=1)    

# clf_1=RandomForestClassifier(criterion='entropy',max_depth= (8), n_estimators=(60),min_samples_split=4,random_state=1)


In [ ]:
# baseline
clf_1.fit(Xtrain,Ytrain)
pred_1 = clf_1.predict(Xtest)
print("auc:",roc_auc_score(Ytest,pred_1))
print("accuracy:",accuracy_score(Ytest,pred_1))
print("recall:",recall_score(Ytest,pred_1))
print("precision:",precision_score(Ytest,pred_1))

auc: 0.9976613657623948
accuracy: 0.9976613657623947
recall: 1.0
precision: 0.9953445065176909


In [14]:
 
clf_2.fit(Xtrain,Ytrain)
pred_2 = clf_2.predict(Xtest)
print("auc:",roc_auc_score(Ytest,pred_2))
print("accuracy:",accuracy_score(Ytest,pred_2))
print("recall:",recall_score(Ytest,pred_2))
print("precision:",precision_score(Ytest,pred_2))

auc: 0.9976613657623948
accuracy: 0.9976613657623947
recall: 1.0
precision: 0.9953445065176909


In [ ]:
clf_3.fit(Xtrain,Ytrain)
pred_3 = clf_3.predict(Xtest)
print("auc:",roc_auc_score(Ytest,pred_3))
print("accuracy:",accuracy_score(Ytest,pred_3))
print("recall:",recall_score(Ytest,pred_3))
print("precision:",precision_score(Ytest,pred_3))

auc: 0.9976613657623948
accuracy: 0.9976613657623947
recall: 1.0
precision: 0.9953445065176909


In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X) 
# Changing data to ndarray format
import numpy as np
X = np.array(X) 
clf_1.fit(X,Y)
clf_2.fit(X,Y)
clf_3.fit(X,Y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=2,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=0.6, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
from sklearn import model_selection 
from mlxtend.classifier import StackingCVClassifier
import numpy as np
RANDOM_SEED = 1
lr = LogisticRegression()

np.random.seed(RANDOM_SEED)
sclf = StackingCVClassifier(classifiers=[clf_1,clf_2, clf_3],use_probas=True,meta_classifier=lr,cv=10)
sclf.fit(np.array(X),np.array(Y))


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


StackingCVClassifier(classifiers=[MLPClassifier(activation='relu', alpha=0.1,
                                                batch_size='auto', beta_1=0.9,
                                                beta_2=0.999,
                                                early_stopping=False,
                                                epsilon=1e-08,
                                                hidden_layer_sizes=10,
                                                learning_rate='constant',
                                                learning_rate_init=0.001,
                                                max_fun=15000, max_iter=500,
                                                momentum=0.9,
                                                n_iter_no_change=10,
                                                nesterovs_momentum=True,
                                                power_t=0.5, random_state=1,
                                                shuffle=True, s...
    

In [ ]:
from sklearn import model_selection 
from mlxtend.classifier import StackingCVClassifier
import numpy as np
RANDOM_SEED = 1
lr = LogisticRegression()

np.random.seed(RANDOM_SEED)
sclf = StackingCVClassifier(classifiers=[clf_1, clf_2,clf_3],use_probas=True,meta_classifier=lr,cv=10)
sclf.fit(np.array(Xtrain),np.array(Ytrain))
pred_sclf = sclf.predict(np.array(Xtest))
print("auc:",roc_auc_score(Ytest,pred_sclf))
print("accuracy:",accuracy_score(Ytest,pred_sclf))
print("recall:",recall_score(Ytest,pred_sclf))
print("precision:",precision_score(Ytest,pred_sclf))

auc: 0.9976613657623948
accuracy: 0.9976613657623947
recall: 1.0
precision: 0.9953445065176909


In [ ]:
from xgboost import XGBClassifier
parameters = {'n_estimators': range(30,120,40),'learning_rate':[0.05, 0.1, 0.15],
             'max_depth':range(1,4,2),'random_state':[1] ,'min_child_weight':range(1,4,2), 
#               'reg_lambda':[0.6,0.7],'reg_alpha':[0]
             }
grid_search_xgb = GridSearchCV(XGBClassifier(), parameters, n_jobs=-1,cv=2,
                   scoring='roc_auc' )
grid_result_xgb = grid_search_xgb.fit(Xtrain, Ytrain)
print("Best: %f using %s" % (grid_result_xgb.best_score_,grid_search_xgb.best_params_))
means = grid_result_xgb.cv_results_['mean_test_score']
params = grid_result_xgb.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

Best: 0.998580 using {'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 70, 'random_state': 1}
0.995073  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 30, 'random_state': 1}
0.995073  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 70, 'random_state': 1}
0.995073  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 110, 'random_state': 1}
0.995073  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 30, 'random_state': 1}
0.995073  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 70, 'random_state': 1}
0.995073  with:   {'learning_rate': 0.05, 'max_depth': 1, 'min_child_weight': 3, 'n_estimators': 110, 'random_state': 1}
0.997950  with:   {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 30, 'random_state': 1}
0.997950  with:   {'learning_rate': 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# GBDT
parameters = {
    "learning_rate": [0.02,0.03,0.05,0.07,0.1],
    "subsample":[0.1,0.2,0.3,0.6],
    "max_depth":range(1,6,1),
    'min_samples_split':range(2,5,1),
    'random_state':[1]}
grid_search_GBDT = GridSearchCV(GradientBoostingClassifier(), parameters, n_jobs=-1,cv=2, scoring='roc_auc' )
grid_result_GBDT = grid_search_GBDT.fit(Xtrain, Ytrain)
print("Best: %f using %s" % (grid_result_GBDT.best_score_,grid_search_GBDT.best_params_))
means = grid_result_GBDT.cv_results_['mean_test_score']
params = grid_result_GBDT.cv_results_['params']
for mean,param in zip(means,params):
    print("%f  with:   %r" % (mean,param))

Best: 0.998548 using {'learning_rate': 0.1, 'max_depth': 2, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.6}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.1}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.2}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.3}
0.995073  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 2, 'random_state': 1, 'subsample': 0.6}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 3, 'random_state': 1, 'subsample': 0.1}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 3, 'random_state': 1, 'subsample': 0.2}
0.995060  with:   {'learning_rate': 0.02, 'max_depth': 1, 'min_samples_split': 3, 'random_state': 1, 'subsample': 0.3}
0.995073  with:   {'learning_rate': 0.02, 'max

In [ ]:
# clf_1=DecisionTreeClassifier(max_depth=(8), min_samples_split=33,random_state=1)
# clf_1=GaussianNB()

# dtc = DecisionTreeClassifier(criterion='gini',splitter='random',max_depth=2,min_samples_split=2,random_state=1)
# clf_1=AdaBoostClassifier(base_estimator=dtc,n_estimators=(4))
clf_3=GradientBoostingClassifier(learning_rate=(0.1),max_depth=(5), subsample=0.6,min_samples_split=2,random_state=1) 
clf_2=XGBClassifier( n_estimators= (110),learning_rate=(0.15), max_depth=(3), random_state=1, min_child_weight=1)
# clf_1=XGBClassifier(colsample_bytree=0.2,gamma=0.2,n_estimators= (80),learning_rate=(0.04), max_depth=(3),min_child_weight=2,num_parallel_tree=3,random_state=1,subsample=0.3)
 
# clf_1=RandomForestClassifier(criterion='entropy',max_depth= (8), n_estimators=(60),min_samples_split=4,random_state=1)

In [ ]:
 clf_2.fit(X,Y)
pred_2 = clf_2.predict(Xtest)
print("auc:",roc_auc_score(Ytest,pred_2))
print("accuracy:",accuracy_score(Ytest,pred_2))
print("recall:",recall_score(Ytest,pred_2))
print("precision:",precision_score(Ytest,pred_2))

NameError: ignored

In [ ]:
clf_3.fit(Xtrain,Ytrain)
pred_3 = clf_3.predict(Xtest)
print("auc:",roc_auc_score(Ytest,pred_3))
print("accuracy:",accuracy_score(Ytest,pred_3))
print("recall:",recall_score(Ytest,pred_3))
print("precision:",precision_score(Ytest,pred_3))

auc: 0.9968818210165264
accuracy: 0.9968818210165263
recall: 0.9984409105082632
precision: 0.9953372707491451


auc: 0.976457748674774
accuracy: 0.976457748674774
recall: 0.977860929217337
precision: 0.9751243781094527


In [16]:
# make submission for test public 
test_set=pd.read_csv('/content/drive/MyDrive/2021_s1/dataset/test-public.csv')
test_set= test_set.drop('Id',1) 
test_set

,Source,Sink
0,0,2917
1,0,2956
2,1,4038
3,2,1848
4,3,513
...,...,...
1995,3865,3924
1996,3917,4025
1997,3922,3947
1998,3955,3987


In [ ]:
# # Generating test vectors
# def generate_testset(model,test_set):
      
#     X = []   
#     for index, row in test_set.iterrows(): 
#         X_vec = list(model[str(row[0])])
#         X_vec.extend(model[str(row[1])])
#         X.append(X_vec) 
#     return X 

In [17]:
 
# new
# Generating test vectors using feaute
def generate_testset_using_statistics( test_set):
      
    X = []   
    for index, row in test_set.iterrows(): 
        # try:
        #   common_ne = len(sorted(net.common_neighbors(G, int(row[0]), int(row[1]))))
        # except:
        #   common_ne = len(sorted(net.common_neighbors(G, int(row[1]), int(row[0]))))
        try:
          try:
            n_shortest_path = len( (net.shortest_path(G, int(row[0]), int(row[1]))))
          except:
            n_shortest_path = len( (net.shortest_path(G, int(row[1]), int(row[0]))))
        except:
          n_shortest_path = 0

        X_vec = []
        # X_vec.append(common_ne)
        X_vec.append(n_shortest_path)
        # X_vec = list(model[str(row[0])])
        # X_vec.extend(model[str(row[1])])
        X.append(X_vec) 
    return X 
 

In [18]:
# new
# Generating test vectors using feaute
def generate_testset_using_statistics( test_set):
      
    X = []   
    for index, row in test_set.iterrows(): 
        try:
          common_ne = len(sorted(net.common_neighbors(G, int(row[0]), int(row[1]))))
        except:
          common_ne = len(sorted(net.common_neighbors(G, int(row[1]), int(row[0]))))
        try:
          try:
            n_shortest_path = len( (net.shortest_path(G, int(row[0]), int(row[1]))))
          except:
            n_shortest_path = len( (net.shortest_path(G, int(row[1]), int(row[0]))))
        except:
          n_shortest_path = 0
            
        jaccard_coefficient_p = None
        try:
          jaccard_coefficient_ps = net.jaccard_coefficient(G,[(int(row[0]), int(row[1]))])  

          for u, v, p in jaccard_coefficient_ps:
            jaccard_coefficient_p = p
        except:
          jaccard_coefficient_ps = net.jaccard_coefficient(G,[(int(row[1]), int(row[0]))])  
          for u, v, p in jaccard_coefficient_ps:
            jaccard_coefficient_p = p
 
 
        ra = None
        try:
          resource_allocations = net.resource_allocation_index(G,[(int(row[0]), int(row[1]))])  

          for u, v, p in resource_allocations:
            ra = p
        except:
          resource_allocations = net.resource_allocation_index(G,[(int(row[1]), int(row[0]))])  
          for u, v, p in resource_allocations:
            ra = p

        aa = None
        try:
          adamic_adar_indexs = net.adamic_adar_index(G,[(int(row[1]), int(row[0]))])  

          for u, v, p in adamic_adar_indexs:
            aa = p
        except:
          adamic_adar_indexs = net.adamic_adar_index(G,[(int(row[0]), int(row[1]))])  
          for u, v, p in adamic_adar_indexs:
            aa = p

        X_vec = []
        X_vec.append(common_ne)
        X_vec.append(n_shortest_path)
        X_vec.append(jaccard_coefficient_p)
        X_vec.append(ra)
        X_vec.append(aa)

        # X_emb = (list(model[str(row[0])]))
        # X_emb.extend(model[str(row[1])]) 
        # X_vec = X_vec + X_emb 
        X.append(X_vec) 
    return X  

In [19]:
import math
# Making prediction for Kaggle test set
submit_test = np.array(generate_testset_using_statistics( test_set))


In [20]:
submit_test

array([[0.        , 4.        , 0.        , 0.        , 0.        ],
       [0.        , 8.        , 0.        , 0.        , 0.        ],
       [0.        , 5.        , 0.        , 0.        , 0.        ],
       ...,
       [0.        , 4.        , 0.        , 0.        , 0.        ],
       [2.        , 2.        , 0.06896552, 0.41666667, 1.31266883],
       [0.        , 7.        , 0.        , 0.        , 0.        ]])

In [21]:
submit_test.shape

(2000, 5)

In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
submit_test = sc.fit_transform(submit_test) 
submit_test

array([[-0.44157272, -0.03446984, -0.34380463, -0.35981212, -0.4092013 ],
       [-0.44157272,  1.72195509, -0.34380463, -0.35981212, -0.4092013 ],
       [-0.44157272,  0.40463639, -0.34380463, -0.35981212, -0.4092013 ],
       ...,
       [-0.44157272, -0.03446984, -0.34380463, -0.35981212, -0.4092013 ],
       [ 1.94530686, -0.91268231,  0.13205006,  2.82905814,  2.5413056 ],
       [-0.44157272,  1.28284886, -0.34380463, -0.35981212, -0.4092013 ]])

In [23]:
# Making prediction on the testing set 
pred_list=[i[1] for i in clf_3.predict_proba(submit_test)]
pred_2d=[]
for index in range(len(pred_list)): 
    pred_2d.append([index+1, pred_list[index]])

test_data_pred = pd.DataFrame(pred_2d, columns=['Id', 'Predicted'])
test_data_pred


,Id,Predicted
0,1,0.000792
1,2,0.000792
2,3,0.000792
3,4,0.998359
4,5,0.000792
...,...,...
1995,1996,0.000792
1996,1997,0.000779
1997,1998,0.000792
1998,1999,0.999161


In [24]:
test_data_pred.to_csv('/content/drive/MyDrive/2021_s1/hongzhi/submit.csv', index=None)